In [86]:
import sys
sys.path.append("/home/aiuser/work/userlib/BigQuant_Resources_Collection/BigQuant_Resources_Collection/00_General_Resources/General_Tool")
sys.path.append("/home/aiuser/work/userlib/BigQuant_Resources_Collection/BigQuant_Resources_Collection/00_General_Resources/General_Strategy")
from cn_general_tool import *
from cn_stock_strategy_factor import *

In [87]:
sd, ed = get_sd_ed('2023-01-01', '2024-11-25')

In [88]:
sql = f"""
WITH
data_industry AS (
    WITH
    data_industry_1 AS (
        SELECT DISTINCT
            date,
            sw2021_level1,
            SUM(float_market_cap) OVER (PARTITION BY date, sw2021_level1) AS industry_market_cap,
        FROM cn_stock_prefactors
    ),
    data_industry_2 AS (
        SELECT
            date,
            sw2021_level1,
            industry_market_cap / LAG(industry_market_cap,  5) OVER (PARTITION BY sw2021_level1 ORDER BY date) AS ret5,
            industry_market_cap / LAG(industry_market_cap, 20) OVER (PARTITION BY sw2021_level1 ORDER BY date) AS ret20,
            industry_market_cap / LAG(industry_market_cap, 60) OVER (PARTITION BY sw2021_level1 ORDER BY date) AS ret60,
            ret5 * 0.4 + ret20 * 0.3 + ret60 * 0.3 AS industry_momt,
        FROM data_industry_1
    )
    SELECT 
        date,
        sw2021_level1,
        RANK() OVER (PARTITION BY date ORDER BY industry_momt DESC) AS industry_momt_rank
    FROM data_industry_2
),
data_stock AS (
    SELECT
        date,
        instrument,
        sw2021_level1,
        c_pct_rank(roe_avg_lf) + c_pct_rank(net_profit_to_parent_qoq_mrq) + c_pct_rank(-1*pb) as score,
        RANK() OVER (PARTITION BY date, sw2021_level1 ORDER BY score DESC) AS score_rank,
    FROM cn_stock_prefactors
),
data_merge AS (
    SELECT 
        date,
        instrument,
        sw2021_level1,
        score,
        score_rank,
        industry_momt_rank,
    FROM data_stock JOIN data_industry USING (date, sw2021_level1)
    QUALIFY industry_momt_rank <= 3 AND score_rank <= 3
    ORDER BY date, instrument
)
SELECT 
    date,
    instrument,
    score AS factor
FROM data_merge 
ORDER BY date
"""

In [89]:
df = get_dai_df(sql, sd, ed)

In [90]:
def BigTrader_Initialize(context):
    from bigtrader.finance.commission import PerOrder
    context.set_commission(PerOrder(buy_cost=0.0003, sell_cost=0.0013, min_cost=5))

def BigTrader_Before_Trading(context, data):
    pass

def BigTrader_Handle_Tick(context, tick):
    pass

def BigTrader_Handle_Data(context, data):

    df_now = context.data[context.data["date"] == data.current_dt.strftime("%Y-%m-%d")]

    if len(df_now) == 0:
        return
    
    target_instruments  = set(list(df_now["instrument"]))
    holding_instruments = set(list(context.get_account_positions().keys()))

    for instrument in holding_instruments - target_instruments:
        context.order_target_percent(instrument, 0)
        
    for instrument in target_instruments - holding_instruments:
        context.order_target_percent(instrument, 0.1)

def BigTrader_Handle_Trade(context, trade):
    pass

def BigTrader_Handle_Order(context, order):
    pass

def BigTrader_After_Trading(context, data):
    pass

BigTrader = M.bigtrader.v34(
    
    data = df,
    
    start_date = """""",
    end_date   = """""",
    
    initialize           = BigTrader_Initialize,
    before_trading_start = BigTrader_Before_Trading,
    handle_tick          = BigTrader_Handle_Tick,
    handle_data          = BigTrader_Handle_Data,
    handle_trade         = BigTrader_Handle_Trade,
    handle_order         = BigTrader_Handle_Order,
    after_trading        = BigTrader_After_Trading,
    
    capital_base = 1000000,
    frequency="""daily""",
    product_type="""自动""",
    rebalance_period_type="""交易日""",
    rebalance_period_days="""1""",
    rebalance_period_roll_forward=True,
    backtest_engine_mode="""标准模式""",
    before_start_days=0,
    volume_limit=1,
    order_price_field_buy="""close""",
    order_price_field_sell="""close""",
    benchmark="""沪深300指数""",
    
    plot_charts=True,
    debug=False,
    backtest_only=False,
    m_name="""BigTrader"""
) 

[2024-12-02 15:13:22] [info     ] bigtrader.v34 开始运行 ..


[2024-12-02 15:13:22] [info     ] 2023-01-03, 2024-11-25, , , instruments=360
[2024-12-02 15:13:22] [info     ] bigtrader module V2.2.0
[2024-12-02 15:13:22] [info     ] bigtrader engine v1.10.10 2024-11-26


INFO:MAIN:======== bigtrader pid:2199 version 1.10.10 2024-11-26 ========
INFO:MAIN:bigtrader run_mode:BACKTEST, handle_bar_mode:0, frequency:1d, exchange_mode:BQ2
INFO:MAIN:> process add account:BACKTEST,0,bkt000
INFO:ACCT[bkt000]:AccountEngine: self_calc:1, validate_self_trading:0, validate_cash:0, validate_position:1, enable_auto_planed_order:1
INFO:MAIN:login_account(bkt000)
INFO:MAIN:> add_strategy setting:{'strategy_name': 'strategy', 'account_id': 'bkt000'}
INFO:MAIN:init all strategy account_id:...
INFO:MAIN:stop all strategy account_id:...


[2024-12-02 15:13:35] [info     ] backtest done, raw_perf_ds:dai.DataSource("_ec56e1324f074ea898d7626cca6e626f")


[2024-12-02 15:13:37] [info     ] bigtrader.v34 运行完成 [15.117s].
